In [1]:
# Imports
import pandas as pd
from pathlib import Path
from collections import Counter


In [2]:
SPOTIFY_DATA_FOLDER_DIR = r"C:\Users\JM070903\OneDrive - Jacobs\Documents\Python\Spotify Listening Analysis\Spotify_Analysis_2.0\1. Data\MyData"

In [82]:
tracks_to_remove = [
    "Binaural Beta Sinus Drone II",
    "Binaural Alpha Sinus 100 Hz - 108 Hz",
    "Cabin Sound",
    "Unknown Track",
]  # Most of these songs are white noise I listen to on repeat, need to remove them from the df
artists_to_remove = [
    "Unknown Artist",
    "Stuff You Should Know",
    "Freakonomics Radio",
    "World War One",
    "The History of WWII Podcast - by Ray Harris Jr",
]  # Removing podcasts and unknown tracks

In [83]:
def gather_mydata(folder_dir):
    """
    Returns consolidated dataframe of listening history from Spotify .json files

    Parameters:
    folder_dir (string): folder directory location of .json Spotify files
    
    Returns:

    spotify_data_df (pandas dataframe): Dataframe object containing combined and consolidated listening history
    """

    _folder_dir = Path(folder_dir)  # converting to pathlib object for OS-agnostic manipulation
    _jsons = Path(_folder_dir).glob("*.json")  # Finding all files in the json folder that end with .json -> generator object
    _json_list = [file.name for file in _jsons]  # retrieving filename of .json files only
    _streaming_list = [s for s in _json_list if "StreamingHistory" in s] #grabbing .jsons for streaming history only
    _spotify_data = {key: [] for key in _streaming_list} # Creating empty dict with json filenames as keys

    for spotify_json in _streaming_list:
        json_filepath = (Path(_folder_dir, spotify_json))    
        read_data = pd.read_json(json_filepath, typ="series", encoding="utf8")
        _spotify_data[spotify_json].append(read_data)

    streams_list = [key[0] for key in _spotify_data.values()]

    spotify_data_df = pd.concat(streams_list, ignore_index=True, sort=False)
    spotify_data_df = pd.json_normalize(spotify_data_df)  # This is a really handy way of converting dict keys to column names
   
    return spotify_data_df

In [127]:
class CleanDataFrame:
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def remove_track(self, tracknames):
        """
        Removes any tracks passed to argument
        
        Parameters:
        tracknames (list): list of tracks to remove
        
        Returns:
        self: original dataframe object with tracks removed
        
        """
        for track in tracknames:
            self.dataframe = self.dataframe[
                ~self.dataframe["trackName"].str.contains(track)
            ]
        return self 

    def remove_artist(self, artists):
        """
        Removes any artists passed to argument
        
        Parameters:
        artists (list): list of artists to remove
        
        Returns:
        self: original dataframe object with artists removed
        
        """
        for artist in artists:
            self.dataframe = self.dataframe[
                ~self.dataframe["artistName"].str.contains(artist)
            ]
        return self

    def convert_datetime(self):
        """
        Converts all time based df columns to datetime objects
        
        Parameters:
        self (dataframe): original dataframe to be cleaned
        
        Returns:
        self (dataframe): cleaned dataframe
        """
        self.dataframe["endTime"] = pd.to_datetime(
            self.dataframe["endTime"], format="%Y-%m-%d %H:%M"
        )
        self.dataframe["minutesPlayed"] = self.dataframe["msPlayed"].divide(60000)

        self.dataframe.drop(["endTime", "msPlayed"], axis=1, inplace=True)

        return self
    
    def remove_duplicates(self):
        """
        Combines the play time of duplicate tracks into a single total play time track
        
        Parameters:
        self (dataframe): original dataframe to have duplicates combined
        
        Returns:
        self (dataframe): cleaned dataframe with duplicate song data combined
        """
        self.dataframe["minutesTotal"] = self.dataframe.groupby(by=["trackName"])["minutesPlayed"].transform("sum")
        self.dataframe.drop_duplicates(subset=["trackName"]).drop(["minutesPlayed"], axis=1)
        
        return self
        

    def return_df(self): #ToDo - shouldn't need this method. The dataframe shoud be returned when any of the above methods are called. 
        """ Returns the dataframe object, rather than a CleanDataFrame object. """
        return self.dataframe


In [128]:
streams_df = gather_mydata(SPOTIFY_DATA_FOLDER_DIR);

In [129]:
clean_data = CleanDataFrame(streams_df)
streams_df = clean_data.remove_track(tracks_to_remove).remove_artist(artists_to_remove).convert_datetime().remove_duplicates().return_df()

In [130]:
streams_df

,artistName,trackName,minutesPlayed,minutesTotal
2,The Growlers,California,0.060100,26.872367
3,Richard Thompson,1952 Vincent Black Lightning,0.004733,2.934550
5,The Growlers,California,1.190883,26.872367
6,Dire Straits,Skateaway,6.646000,9.568033
7,Doc Robinson,Break My Fall,4.801550,45.732567
...,...,...,...,...
30716,Floating,Gamma Colours,3.386133,28.395400
30717,Floating,Gamma Colours,1.317200,28.395400
30752,Fazerdaze,Little Uneasy,1.936250,9.537450
30754,Slowdive,Star Roving,0.728033,2.463633


In [131]:
def save_dataframe(cleaned_dataframe, save_directory, filename):
    """
    Saves the cleaned dataframe as a pickle file 

    Parameters:
    cleaned_dataframe (dataframe): pandas dataframe object to be saved
    save_directory (string): folder directory of where cleaned dataframe .pkl is to be saved
    filename (string): name of .pkl file to be saved

    Returns:
    saved_pickle (.pkl file): .pkl file of dataframe, saved in user submitted save directory, with chosen filename
    """

    _save_directory = Path(save_directory + filename  + ".pkl")
    return cleaned_dataframe.to_pickle(_save_directory)

In [132]:
save_directory = r"C:\Users\JM070903\OneDrive - Jacobs\Documents\Python\Spotify Listening Analysis\Spotify_Analysis_2.0\2. PreProcessing\PreProcessing_MyData"
filename = "listening_history"

In [135]:
Path(save_directory, filename  + ".pkl")

WindowsPath('C:/Users/JM070903/OneDrive - Jacobs/Documents/Python/Spotify Listening Analysis/Spotify_Analysis_2.0/2. PreProcessing/PreProcessing_MyData/listening_history.pkl')

In [ ]:
(dir_name, base_filename + suffix)